In [1]:
from functools import partial

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 400)
pd.set_option('display.max_rows', 30)

from server2022.data.processed.prepare_data import create_df_2years_known, create_df_0years_known, create_df_1year_known
from server2022.data.processed.prepare_data import factors_2020, FINANCE_FEAT

In [3]:
df_2021 = pd.read_csv('../server2022/data/raw/agents2021.csv')

In [4]:
df_2021.columns.to_list()

['Unnamed: 0',
 'Наименование ДП',
 'Макс. ПДЗ за 2019 год, дней',
 'Сред. ПДЗ за 2019 год, дней',
 'Кол-во просрочек свыше 5-ти дней за 2019 год, шт.',
 'Общая сумма ПДЗ свыше 5-ти дней за 2019 год, руб.',
 'Кол-во раз ПДЗ за 2019 год, шт.',
 'Макс. ПДЗ за 2020 год, дней',
 'Сред. ПДЗ за 2020 год, дней',
 'Кол-во просрочек свыше 5-ти дней за 2020 год, шт.',
 'Общая сумма ПДЗ свыше 5-ти дней за 2020 год, руб.',
 'Кол-во раз ПДЗ за 2020 год, шт.',
 'Оценка потенциала контрагента 1, руб.',
 'Оценка потенциала контрагента 2, руб.',
 'Статус',
 '2016, Нематериальные активы, RUB',
 '2017, Нематериальные активы, RUB',
 '2018, Нематериальные активы, RUB',
 '2019, Нематериальные активы, RUB',
 '2020, Нематериальные активы, RUB',
 '2016, Основные средства , RUB',
 '2017, Основные средства , RUB',
 '2018, Основные средства , RUB',
 '2019, Основные средства , RUB',
 '2020, Основные средства , RUB',
 '2016, Внеоборотные активы, RUB',
 '2017, Внеоборотные активы, RUB',
 '2018, Внеоборотные активы, 

In [5]:
col = '2020, Выручка, RUB'

In [8]:
cols = [x for x in df_2021.columns if 'Выручка' in x]
cols

['2016, Выручка, RUB',
 '2017, Выручка, RUB',
 '2018, Выручка, RUB',
 '2019, Выручка, RUB',
 '2020, Выручка, RUB']

In [46]:
for col in df_2021.columns:
    flag = False
    for f in FINANCE_FEAT:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    print(f'"{col}"', ':', (df_2021[col].abs() > 10).sum(), '--', (df_2021[col].abs() < 10).sum())

"2016, Нематериальные активы, RUB" : 86 -- 239
"2017, Нематериальные активы, RUB" : 90 -- 235
"2018, Нематериальные активы, RUB" : 103 -- 222
"2019, Нематериальные активы, RUB" : 112 -- 213
"2020, Нематериальные активы, RUB" : 115 -- 210
"2016, Основные средства , RUB" : 250 -- 75
"2017, Основные средства , RUB" : 254 -- 71
"2018, Основные средства , RUB" : 291 -- 34
"2019, Основные средства , RUB" : 306 -- 19
"2020, Основные средства , RUB" : 307 -- 18
"2016, Внеоборотные активы, RUB" : 253 -- 72
"2017, Внеоборотные активы, RUB" : 261 -- 64
"2018, Внеоборотные активы, RUB" : 293 -- 32
"2019, Внеоборотные активы, RUB" : 309 -- 16
"2020, Внеоборотные активы, RUB" : 308 -- 17
"2016, Дебиторская задолженность, RUB" : 267 -- 58
"2017, Дебиторская задолженность, RUB" : 276 -- 49
"2018, Дебиторская задолженность, RUB" : 307 -- 18
"2019, Дебиторская задолженность, RUB" : 320 -- 5
"2020, Дебиторская задолженность, RUB" : 318 -- 7
"2016, Оборотные активы, RUB" : 270 -- 55
"2017, Оборотные актив

In [58]:
fin_feats = [
    "2019, Основные средства , RUB",
    "2020, Основные средства , RUB",
    "2019, Внеоборотные активы, RUB",
    "2020, Внеоборотные активы, RUB",
    "2019, Дебиторская задолженность, RUB",
    "2020, Дебиторская задолженность, RUB",
    "2019, Оборотные активы, RUB",
    "2020, Оборотные активы, RUB",
    "2019, Капитал и резервы, RUB",
    "2020, Капитал и резервы, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
    "2019, Прибыль (убыток) до налогообложения , RUB",
    "2020, Прибыль (убыток) до налогообложения , RUB",
    "2019, Прибыль (убыток) от продажи, RUB",
    "2019, Прибыль (убыток) от продажи, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col].abs() < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

29

In [ ]:
2018

In [61]:
fin_feats = [
    "2018, Основные средства , RUB",
    "2019, Основные средства , RUB",
    "2020, Основные средства , RUB",
    
    "2018, Внеоборотные активы, RUB",
    "2019, Внеоборотные активы, RUB",
    "2020, Внеоборотные активы, RUB",
    
    "2018, Дебиторская задолженность, RUB",
    "2019, Дебиторская задолженность, RUB",
    "2020, Дебиторская задолженность, RUB",
    
    "2018, Оборотные активы, RUB",
    "2019, Оборотные активы, RUB",
    "2020, Оборотные активы, RUB",
    
    "2018, Капитал и резервы, RUB",
    "2019, Капитал и резервы, RUB",
    "2020, Капитал и резервы, RUB",
    
    "2018, Кредиторская задолженность, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
    
    "2019, Краткосрочные обязательства, RUB",
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    
    "2018, Выручка, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    
    "2018, Себестоимость продаж, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
    
    "2018, Кредиторская задолженность, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
    
    "2018, Прибыль (убыток) до налогообложения , RUB",
    "2019, Прибыль (убыток) до налогообложения , RUB",
    "2020, Прибыль (убыток) до налогообложения , RUB",
    
    "2018, Прибыль (убыток) от продажи, RUB",
    "2019, Прибыль (убыток) от продажи, RUB",
    "2019, Прибыль (убыток) от продажи, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col].abs() < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

47

In [59]:
fin_feats = [
    "2019, Основные средства , RUB",
    "2020, Основные средства , RUB",
#     "2019, Внеоборотные активы, RUB",
#     "2020, Внеоборотные активы, RUB",
    "2019, Дебиторская задолженность, RUB",
    "2020, Дебиторская задолженность, RUB",
    "2019, Оборотные активы, RUB",
    "2020, Оборотные активы, RUB",
    "2019, Капитал и резервы, RUB",
    "2020, Капитал и резервы, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
    "2019, Прибыль (убыток) до налогообложения , RUB",
    "2020, Прибыль (убыток) до налогообложения , RUB",
    "2019, Прибыль (убыток) от продажи, RUB",
    "2019, Прибыль (убыток) от продажи, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col].abs() < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

29

In [60]:
fin_feats = [
#     "2019, Основные средства , RUB",
#     "2020, Основные средства , RUB",
    "2019, Внеоборотные активы, RUB",
    "2020, Внеоборотные активы, RUB",
    "2019, Дебиторская задолженность, RUB",
    "2020, Дебиторская задолженность, RUB",
    "2019, Оборотные активы, RUB",
    "2020, Оборотные активы, RUB",
    "2019, Капитал и резервы, RUB",
    "2020, Капитал и резервы, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
    "2019, Прибыль (убыток) до налогообложения , RUB",
    "2020, Прибыль (убыток) до налогообложения , RUB",
    "2019, Прибыль (убыток) от продажи, RUB",
    "2019, Прибыль (убыток) от продажи, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col].abs() < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

27

In [57]:
fin_feats = [
#     "2019, Внеоборотные активы, RUB",
#     "2020, Внеоборотные активы, RUB",
    "2019, Дебиторская задолженность, RUB",
    "2020, Дебиторская задолженность, RUB",
    "2019, Оборотные активы, RUB",
    "2020, Оборотные активы, RUB",
    "2019, Капитал и резервы, RUB",
    "2020, Капитал и резервы, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
    "2019, Прибыль (убыток) до налогообложения , RUB",
    "2020, Прибыль (убыток) до налогообложения , RUB",
    "2019, Прибыль (убыток) от продажи, RUB",
    "2019, Прибыль (убыток) от продажи, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col].abs() < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

14

In [53]:
fin_feats = [
#     "2019, Внеоборотные активы, RUB",
#     "2020, Внеоборотные активы, RUB"
    "2019, Дебиторская задолженность, RUB",
    "2020, Дебиторская задолженность, RUB",
    "2019, Капитал и резервы, RUB",
    "2020, Капитал и резервы, RUB",
    "2019, Прибыль (убыток) до налогообложения , RUB",
    "2020, Прибыль (убыток) до налогообложения , RUB",
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col].abs() < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

14

In [50]:
fin_feats = [
    "2019, Прибыль (убыток) до налогообложения , RUB",
    "2020, Прибыль (убыток) до налогообложения , RUB",
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col] < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

55

In [40]:
fin_feats = [
    "2019, Дебиторская задолженность, RUB",
    "2020, Дебиторская задолженность, RUB"
    "2019, Капитал и резервы, RUB",
    "2020, Капитал и резервы, RUB",
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col] < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

23

In [41]:
q = set(bad_ids)

In [45]:
fin_feats = [
    "2019, Капитал и резервы, RUB",
    "2020, Капитал и резервы, RUB",
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col].abs() < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

11

In [43]:
set(bad_ids) - q

{185, 304, 499}

In [44]:
df_2021[df_2021['Наименование ДП'] == 185]

,Unnamed: 0,Наименование ДП,"Макс. ПДЗ за 2019 год, дней","Сред. ПДЗ за 2019 год, дней","Кол-во просрочек свыше 5-ти дней за 2019 год, шт.","Общая сумма ПДЗ свыше 5-ти дней за 2019 год, руб.","Кол-во раз ПДЗ за 2019 год, шт.","Макс. ПДЗ за 2020 год, дней","Сред. ПДЗ за 2020 год, дней","Кол-во просрочек свыше 5-ти дней за 2020 год, шт.","Общая сумма ПДЗ свыше 5-ти дней за 2020 год, руб.","Кол-во раз ПДЗ за 2020 год, шт.","Оценка потенциала контрагента 1, руб.","Оценка потенциала контрагента 2, руб.",Статус,"2016, Нематериальные активы, RUB","2017, Нематериальные активы, RUB","2018, Нематериальные активы, RUB","2019, Нематериальные активы, RUB","2020, Нематериальные активы, RUB","2016, Основные средства , RUB","2017, Основные средства , RUB","2018, Основные средства , RUB","2019, Основные средства , RUB","2020, Основные средства , RUB","2016, Внеоборотные активы, RUB","2017, Внеоборотные активы, RUB","2018, Внеоборотные активы, RUB","2019, Внеоборотные активы, RUB","2020, Внеоборотные активы, RUB","2016, Дебиторская задолженность, RUB","2017, Дебиторская задолженность, RUB","2018, Дебиторская задолженность, RUB","2019, Дебиторская задолженность, RUB","2020, Дебиторская задолженность, RUB","2016, Оборотные активы, RUB","2017, Оборотные активы, RUB","2018, Оборотные активы, RUB","2019, Оборотные активы, RUB","2020, Оборотные активы, RUB","2016, Уставный капитал , RUB","2017, Уставный капитал , RUB","2018, Уставный капитал , RUB","2019, Уставный капитал , RUB","2020, Уставный капитал , RUB","2016, Капитал и резервы, RUB","2017, Капитал и резервы, RUB","2018, Капитал и резервы, RUB","2019, Капитал и резервы, RUB","2020, Капитал и резервы, RUB","2016, Заёмные средства (долгосрочные), RUB","2017, Заёмные средства (долгосрочные), RUB","2018, Заёмные средства (долгосрочные), RUB","2019, Заёмные средства (долгосрочные), RUB","2020, Заёмные средства (долгосрочные), RUB","2016, Долгосрочные обязательства, RUB","2017, Долгосрочные обязательства, RUB","2018, Долгосрочные обязательства, RUB","2019, Долгосрочные обязательства, RUB","2020, Долгосрочные обязательства, RUB","2016, Заёмные средства (краткосрочные), RUB","2017, Заёмные средства (краткосрочные), RUB","2018, Заёмные средства (краткосрочные), RUB","2019, Заёмные средства (краткосрочные), RUB","2020, Заёмные средства (краткосрочные), RUB","2016, Кредиторская задолженность, RUB","2017, Кредиторская задолженность, RUB","2018, Кредиторская задолженность, RUB","2019, Кредиторская задолженность, RUB","2020, Кредиторская задолженность, RUB","2016, Краткосрочные обязательства, RUB","2017, Краткосрочные обязательства, RUB","2018, Краткосрочные обязательства, RUB","2019, Краткосрочные обязательства, RUB","2020, Краткосрочные обязательства, RUB","2016, Выручка, RUB","2017, Выручка, RUB","2018, Выручка, RUB","2019, Выручка, RUB","2020, Выручка, RUB","2016, Себестоимость продаж, RUB","2017, Себестоимость продаж, RUB","2018, Себестоимость продаж, RUB","2019, Себестоимость продаж, RUB","2020, Себестоимость продаж, RUB","2016, Прибыль (убыток) до налогообложения , RUB","2017, Прибыль (убыток) до налогообложения , RUB","2018, Прибыль (убыток) до налогообложения , RUB","2019, Прибыль (убыток) до налогообложения , RUB","2020, Прибыль (убыток) до налогообложения , RUB","2016, Прибыль (убыток) от продажи, RUB","2017, Прибыль (убыток) от продажи, RUB","2018, Прибыль (убыток) от продажи, RUB","2019, Прибыль (убыток) от продажи, RUB","2020, Прибыль (убыток) от продажи, RUB",Факт. 20,Факт. 21,Факт.32,Факт.31,Факт.23,Факт 24,Факт 27,Факт 33,Факт 28,Факт 29,Факт 30,Факт 40,Факт 41,Факт 42,Факт 46,Факт 48,Факт 49,Факт 50,Факт 51,Факт 54,Факт 55,Факт 56,Факт 57,Факт 58,Факт 59,Факт 60,Факт 37,Факт 39,Факт 1,Факт 2,Факт 3,Факт 7,Факт 12,Факт 14,Факт 15,Факт 16,Итого,ПДЗ 1-30,ПДЗ 31-90,ПДЗ 91-365,ПДЗ более 365
114,114,185,8,1.944444,2,546157.292994,18,265,95.0,3,157146.566879,3,1.613503e+07,1.613503e+07,Действующая,7.751592e+06,1.489873e+07,4.429299e+06,2.177070e+06,1.307707e+07,9.243688e+08,8.818191e+08,2.057803e+

In [31]:
fin_feats = [
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
    "2019, Кредиторская задолженность, RUB",
    "2020, Кредиторская задолженность, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col] < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

11

In [30]:
fin_feats = [
    "2019, Краткосрочные обязательства, RUB",
    "2020, Краткосрочные обязательства, RUB",
    "2019, Выручка, RUB",
    "2020, Выручка, RUB",
    "2019, Себестоимость продаж, RUB",
    "2020, Себестоимость продаж, RUB",
]

bad_ids = []
for col in df_2021.columns:
    flag = False
    for f in fin_feats:
        if f in col:
            flag = True
            break
    if not flag:
        continue
    
    bad_ids.extend(df_2021[df_2021[col] < 10]['Наименование ДП'].values.tolist())
    
len(set(bad_ids))

11

In [13]:
(df_2021[cols] < 10).any(axis=1).sum()

79

In [15]:
df_2021[(df_2021[cols] < 10).any(axis=1)][cols]

,"2016, Выручка, RUB","2017, Выручка, RUB","2018, Выручка, RUB","2019, Выручка, RUB","2020, Выручка, RUB"
6,9.243885e+08,0.000000e+00,1.031372e+09,1.157608e+09,1.415543e+09
34,2.861519e+10,1.725111e+10,0.000000e+00,0.000000e+00,0.000000e+00
37,6.760121e+09,4.734053e+09,1.104855e+10,0.000000e+00,0.000000e+00
40,4.730851e+10,5.655227e+10,6.840285e+10,7.456315e+10,0.000000e+00
41,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...
319,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
320,0.000000e+00,0.000000e+00,0.000000e+00,1.639618e+08,3.689191e+08
321,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
322,0.000000e+00,0.000000e+00,0.000000e+00,4.092166e+07,3.399936e+07
